In [1]:
# #Making new smaller .csv

# import pandas as pd
# df=pd.read_csv("static/Files/Result_Data.csv")
# del df['IATA_Airline']
# del df['FlightNumber']
# del df['TailNumber']
# del df['CRSDepTime']
# del df['CRSArrTime']
# del df['CRSElapsedTime']
# del df['Distance']
# del df['CancellationCode']
# del df['DepTime']
# del df['ArrTime']
# del df['ActualElapsedTime']
# del df['AirTime']
# del df['TaxiOut']
# del df['WheelsOff']
# del df['WheelsOn']
# del df['TaxiIn']
# df.to_csv("static/Files/ResultDataForAirports.csv",index=False)


In [38]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [4]:
df=pd.read_csv("static/files/ResultDataForAirports.csv")
df

,ID,FlightDate,Origin,Dest,Cancelled,Diverted,DepDelay,ArrDelay,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,1,2015-01-01,JFK,LAX,0,0,-5,7,0,0,0,0,0
1,2,2015-01-01,LAX,JFK,0,0,-4,-44,0,0,0,0,0
2,3,2015-01-01,JFK,LAX,0,0,-4,-2,0,0,0,0,0
3,4,2015-01-01,LAX,JFK,0,0,-6,-17,0,0,0,0,0
4,5,2015-01-01,DFW,HNL,0,0,289,300,197,0,11,0,92
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31746836,31746837,2019-12-31,TUS,LAX,0,0,33,3,0,0,0,0,0
31746837,31746838,2019-12-31,TUS,MDW,0,0,-6,-26,0,0,0,0,0
31746838,31746839,2019-12-31,TUS,SAN,0,0,16,8,0,0,0,0,0
31746839,31746840,2019-12-31,TUS,SAN,0,0,-3,-22,0,0,0,0,0


In [101]:
listOfAirport=df.Origin.unique()
#print(listOfAirport)
#print(len(listOfAirport))
#print(len(df.Dest.unique()))

371
369


In [103]:
import numpy as np
def createAirportCSV(AirportCode):
    #Yearly Calculations
    dep_filter=df['Origin']==AirportCode
    dep_df=df[dep_filter]
    dep_df['FlightDate'] = pd.to_datetime(df['FlightDate'], errors='coerce')
    dep_df['Year'] = dep_df['FlightDate'].dt.year
    
    arr_filter=df['Dest']==AirportCode
    arr_df=df[arr_filter]
    arr_df['FlightDate'] = pd.to_datetime(df['FlightDate'], errors='coerce')
    arr_df['Year'] = arr_df['FlightDate'].dt.year
    
    dep_pt_df1=pd.pivot_table \
    (dep_df, \
     index='Year', \
     values=['ID','DepDelay','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay', \
            'Cancelled','Diverted'], \
     aggfunc={'ID':pd.Series.nunique, 'DepDelay':np.mean, 'CarrierDelay':np.mean, 'WeatherDelay':np.mean, \
              'NASDelay':np.mean,'SecurityDelay':np.mean,'LateAircraftDelay':np.mean, \
             'Cancelled':np.mean,'Diverted':np.mean})    
    dep_pt_df1=dep_pt_df1.rename(columns={"ID": "Count"}, errors="raise")
    #print(dep_pt_df1)
    
    dep_pt_df2=pd.pivot_table \
    (dep_df, \
     index='Year', \
     values=['DepDelay'], \
     aggfunc={'DepDelay': lambda x:(x>=15).sum()})    
    dep_pt_df2=dep_pt_df2.rename(columns={"DepDelay": "15plusDepCount"}, errors="raise")
    #print(dep_pt_df2)
    
    dep_pt_df3=pd.pivot_table \
    (arr_df, \
     index='Year', \
     values=['ID','ArrDelay'], \
     aggfunc={'ID':pd.Series.nunique, 'ArrDelay':np.mean})
    dep_pt_df3=dep_pt_df3.rename(columns={"ID": "Count2"}, errors="raise")
    #print(dep_pt_df3)
    
    dep_pt_df4=pd.pivot_table \
    (arr_df, \
     index='Year', \
     values=['ArrDelay'], \
     aggfunc={'ArrDelay': lambda x:(x>=15).sum()})    
    dep_pt_df4=dep_pt_df4.rename(columns={"ArrDelay": "15plusArrCount"}, errors="raise")
    #print(dep_pt_df4)
    
    #inner join
    merged_df1 = dep_pt_df1.merge(dep_pt_df2, how='inner', left_on=["Year"], right_on=["Year"])
    merged_df2 = dep_pt_df3.merge(dep_pt_df4, how='inner', left_on=["Year"], right_on=["Year"])
    merged_df = merged_df1.merge(merged_df2, how='inner', left_on=["Year"], right_on=["Year"])
    
    #calc 15min+ delay %age
    merged_df['15minDepDelay'] = merged_df['15plusDepCount'] / merged_df['Count']
    merged_df['15minArrDelay'] = merged_df['15plusArrCount'] / merged_df['Count2']
    
    
    #remove useless columns
    del merged_df['15plusDepCount']
    del merged_df['15plusArrCount']
    
    #Type: Yearly=y, Monthly=m, Daily=d
    merged_df['Type']="y"
    merged_df = merged_df.reset_index()
    
    yearly_df=merged_df.rename(columns={"Year": "Time"}, errors="raise")    
    #print(yearly_df)

    
    
    #Monthly Calculations
    dep_filter=df['Origin']==AirportCode
    dep_df=df[dep_filter]
    dep_df['FlightDate'] = pd.to_datetime(df['FlightDate'], errors='coerce')
    dep_df['Month'] = dep_df['FlightDate'].dt.month
    
    arr_filter=df['Dest']==AirportCode
    arr_df=df[arr_filter]
    arr_df['FlightDate'] = pd.to_datetime(df['FlightDate'], errors='coerce')
    arr_df['Month'] = arr_df['FlightDate'].dt.month
    dep_pt_df1=pd.pivot_table \
    (dep_df, \
     index='Month', \
     values=['ID','DepDelay','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay', \
            'Cancelled','Diverted'], \
     aggfunc={'ID':pd.Series.nunique, 'DepDelay':np.mean, 'CarrierDelay':np.mean, 'WeatherDelay':np.mean, \
              'NASDelay':np.mean,'SecurityDelay':np.mean,'LateAircraftDelay':np.mean, \
             'Cancelled':np.mean,'Diverted':np.mean})    
    dep_pt_df1=dep_pt_df1.rename(columns={"ID": "Count"}, errors="raise")
    #print(dep_pt_df1)
    
    dep_pt_df2=pd.pivot_table \
    (dep_df, \
     index='Month', \
     values=['DepDelay'], \
     aggfunc={'DepDelay': lambda x:(x>=15).sum()})    
    dep_pt_df2=dep_pt_df2.rename(columns={"DepDelay": "15plusDepCount"}, errors="raise")
    #print(dep_pt_df2)
    
    dep_pt_df3=pd.pivot_table \
    (arr_df, \
     index='Month', \
     values=['ID','ArrDelay'], \
     aggfunc={'ID':pd.Series.nunique, 'ArrDelay':np.mean})
    dep_pt_df3=dep_pt_df3.rename(columns={"ID": "Count2"}, errors="raise")
    #print(dep_pt_df3)
    
    dep_pt_df4=pd.pivot_table \
    (arr_df, \
     index='Month', \
     values=['ArrDelay'], \
     aggfunc={'ArrDelay': lambda x:(x>=15).sum()})    
    dep_pt_df4=dep_pt_df4.rename(columns={"ArrDelay": "15plusArrCount"}, errors="raise")
    #print(dep_pt_df4)
    
    #inner join
    merged_df1 = dep_pt_df1.merge(dep_pt_df2, how='inner', left_on=["Month"], right_on=["Month"])
    merged_df2 = dep_pt_df3.merge(dep_pt_df4, how='inner', left_on=["Month"], right_on=["Month"])
    merged_df = merged_df1.merge(merged_df2, how='inner', left_on=["Month"], right_on=["Month"])
    
    #calc 15min+ delay %age
    merged_df['15minDepDelay'] = merged_df['15plusDepCount'] / merged_df['Count']
    merged_df['15minArrDelay'] = merged_df['15plusArrCount'] / merged_df['Count2']
    
    
    #remove useless columns
    del merged_df['15plusDepCount']
    del merged_df['15plusArrCount']
    
    #Type: Yearly=y, Monthly=m, Daily=d
    merged_df['Type']="m"
    merged_df = merged_df.reset_index()
    
    monthly_df=merged_df.rename(columns={"Month": "Time"}, errors="raise")    
    #print(monthly_df)

    
    
    
    #Daily Calculations
    dep_filter=df['Origin']==AirportCode
    dep_df=df[dep_filter]
    dep_df['FlightDate'] = pd.to_datetime(df['FlightDate'], errors='coerce')
    dep_df['DoW'] = dep_df['FlightDate'].dt.dayofweek
    
    arr_filter=df['Dest']==AirportCode
    arr_df=df[arr_filter]
    arr_df['FlightDate'] = pd.to_datetime(df['FlightDate'], errors='coerce')
    arr_df['DoW'] = arr_df['FlightDate'].dt.dayofweek
    dep_pt_df1=pd.pivot_table \
    (dep_df, \
     index='DoW', \
     values=['ID','DepDelay','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay', \
            'Cancelled','Diverted'], \
     aggfunc={'ID':pd.Series.nunique, 'DepDelay':np.mean, 'CarrierDelay':np.mean, 'WeatherDelay':np.mean, \
              'NASDelay':np.mean,'SecurityDelay':np.mean,'LateAircraftDelay':np.mean, \
             'Cancelled':np.mean,'Diverted':np.mean})    
    dep_pt_df1=dep_pt_df1.rename(columns={"ID": "Count"}, errors="raise")
    #print(dep_pt_df1)
    
    dep_pt_df2=pd.pivot_table \
    (dep_df, \
     index='DoW', \
     values=['DepDelay'], \
     aggfunc={'DepDelay': lambda x:(x>=15).sum()})    
    dep_pt_df2=dep_pt_df2.rename(columns={"DepDelay": "15plusDepCount"}, errors="raise")
    #print(dep_pt_df2)
    
    dep_pt_df3=pd.pivot_table \
    (arr_df, \
     index='DoW', \
     values=['ID','ArrDelay'], \
     aggfunc={'ID':pd.Series.nunique, 'ArrDelay':np.mean})
    dep_pt_df3=dep_pt_df3.rename(columns={"ID": "Count2"}, errors="raise")
    #print(dep_pt_df3)
    
    dep_pt_df4=pd.pivot_table \
    (arr_df, \
     index='DoW', \
     values=['ArrDelay'], \
     aggfunc={'ArrDelay': lambda x:(x>=15).sum()})    
    dep_pt_df4=dep_pt_df4.rename(columns={"ArrDelay": "15plusArrCount"}, errors="raise")
    #print(dep_pt_df4)
    
    #inner join
    merged_df1 = dep_pt_df1.merge(dep_pt_df2, how='inner', left_on=["DoW"], right_on=["DoW"])
    merged_df2 = dep_pt_df3.merge(dep_pt_df4, how='inner', left_on=["DoW"], right_on=["DoW"])
    merged_df = merged_df1.merge(merged_df2, how='inner', left_on=["DoW"], right_on=["DoW"])
    
    #calc 15min+ delay %age
    merged_df['15minDepDelay'] = merged_df['15plusDepCount'] / merged_df['Count']
    merged_df['15minArrDelay'] = merged_df['15plusArrCount'] / merged_df['Count2']
    
    
    #remove useless columns
    del merged_df['15plusDepCount']
    del merged_df['15plusArrCount']
    
    #Type: Yearly=y, Monthly=m, Daily=d
    merged_df['Type']="d"
    
    merged_df=merged_df.rename(index={0: 'Mon', 1: 'Tue', 2: 'Wed', \
                                      3: 'Thu', 4: 'Fri', 5: 'Sat', 6: 'Sun'})
    
    merged_df = merged_df.reset_index()
    
    daily_df=merged_df.rename(columns={"DoW": "Time"}, errors="raise")    
    #print(daily_df)
    
    merged_df=yearly_df.append(monthly_df, ignore_index=True)
    merged_df=merged_df.append(daily_df, ignore_index=True)
    return merged_df

In [105]:
#ap_df=createAirportCSV('MCI')
#ap_df.to_csv("static/Files/Test.csv",index=False)

c=1
for item in listOfAirport:
    print(c)
    ap_df=createAirportCSV(item)
    ap_df.to_csv("static/files/airports/"+item+".csv",index=False)
    c+=1
    #if(c==3):
    #    break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
